## IMPORTS

In [5]:

import nltk, re
import numpy as np
import pandas as pd
import time
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize

## DATA PREPROCESSING

In [6]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [7]:
# first few tagged sentences
print(wsj[:10])

[[('Pierre', 'NOUN'), ('Vinken', 'NOUN'), (',', '.'), ('61', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), (',', '.'), ('will', 'VERB'), ('join', 'VERB'), ('the', 'DET'), ('board', 'NOUN'), ('as', 'ADP'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('Nov.', 'NOUN'), ('29', 'NUM'), ('.', '.')], [('Mr.', 'NOUN'), ('Vinken', 'NOUN'), ('is', 'VERB'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Elsevier', 'NOUN'), ('N.V.', 'NOUN'), (',', '.'), ('the', 'DET'), ('Dutch', 'NOUN'), ('publishing', 'VERB'), ('group', 'NOUN'), ('.', '.')], [('Rudolph', 'NOUN'), ('Agnew', 'NOUN'), (',', '.'), ('55', 'NUM'), ('years', 'NOUN'), ('old', 'ADJ'), ('and', 'CONJ'), ('former', 'ADJ'), ('chairman', 'NOUN'), ('of', 'ADP'), ('Consolidated', 'NOUN'), ('Gold', 'NOUN'), ('Fields', 'NOUN'), ('PLC', 'NOUN'), (',', '.'), ('was', 'VERB'), ('named', 'VERB'), ('*-1', 'X'), ('a', 'DET'), ('nonexecutive', 'ADJ'), ('director', 'NOUN'), ('of', 'ADP'), ('this', 'DET'), ('British', 'ADJ'), ('industrial', 'ADJ'), ('

In [9]:
# Splitting into train and test

train_set, test_set = train_test_split(wsj,test_size=0.05,random_state=4)

print(len(train_set))
print(len(test_set))
print(train_set[:10])

3718
196
[[('Yesterday', 'NOUN'), ("'s", 'PRT'), ('share', 'NOUN'), ('turnover', 'NOUN'), ('was', 'VERB'), ('well', 'ADV'), ('below', 'ADP'), ('the', 'DET'), ('year', 'NOUN'), ("'s", 'PRT'), ('daily', 'ADJ'), ('average', 'NOUN'), ('of', 'ADP'), ('133.8', 'NUM'), ('million', 'NUM'), ('.', '.')], [('Yet', 'ADV'), ('he', 'PRON'), ('is', 'VERB'), ("n't", 'ADV'), ('in', 'ADP'), ('favor', 'NOUN'), ('of', 'ADP'), ('new', 'ADJ'), ('legislation', 'NOUN'), ('.', '.')], [('Sea', 'NOUN'), ('Containers', 'NOUN'), ('Ltd.', 'NOUN'), ('said', 'VERB'), ('0', 'X'), ('it', 'PRON'), ('might', 'VERB'), ('increase', 'VERB'), ('the', 'DET'), ('price', 'NOUN'), ('of', 'ADP'), ('its', 'PRON'), ('$', '.'), ('70-a-share', 'ADJ'), ('*U*', 'X'), ('buy-back', 'ADJ'), ('plan', 'NOUN'), ('if', 'ADP'), ('*-2', 'X'), ('pressed', 'VERB'), ('*-1', 'X'), ('by', 'ADP'), ('Temple', 'NOUN'), ('Holdings', 'NOUN'), ('Ltd.', 'NOUN'), (',', '.'), ('which', 'DET'), ('*T*-156', 'X'), ('made', 'VERB'), ('an', 'DET'), ('earlier', 'A

In [10]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95575

In [11]:
# tokens 
tokens = [pair[0] for pair in train_tagged_words]
tokens[:10]

['Yesterday',
 "'s",
 'share',
 'turnover',
 'was',
 'well',
 'below',
 'the',
 'year',
 "'s"]

In [12]:
# vocabulary
V = set(tokens)
print(len(V))

12082


In [13]:
# number of tags
T = set([pair[1] for pair in train_tagged_words])
len(T)

12

In [14]:
print(T)

{'X', 'PRT', 'VERB', 'CONJ', 'NOUN', 'NUM', 'ADJ', 'ADP', 'DET', 'ADV', '.', 'PRON'}


In [15]:
#Function Generates a dictionary which consist of Tags as keys and the list of words that are tagged under the specific tag
#This dictionary helps will calculating the emission probability
def tag_list_gen(T,train_bag = train_tagged_words):
    tag_list={}
    for tag in T:
        tag_occur=[pair[0] for pair in train_bag if pair[1]==tag]
        tag_list[tag]=tag_occur
        
    return tag_list

In [16]:
tags_list=tag_list_gen(T)

### Emission Probabilities

In [19]:
# computing P(w/t) and storing in T x V matrix
t = len(T)
v = len(V)
w_given_t = np.zeros((t, v))

In [20]:
# compute word given tag: Emission Probability
def word_given_tag(word, tag, train_bag = train_tagged_words,tags_list=tags_list):
    tag_list = tags_list[tag]
    count_tag = len(tag_list)
    
    count_w_given_tag = tag_list.count(word)
    
    return (count_w_given_tag, count_tag)

### Transition Probabilities

In [21]:
#Generate a list with the tag order to compute Transition Probability
tag_order=[pair[1] for pair in train_tagged_words]

In [22]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words,tags=tag_order):
    
    count_t1 = len(tags_list[t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [23]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)
start=time.time()
tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]
end=time.time()
print(end-start)

3.464737892150879


In [24]:
tags_matrix

array([[7.51665086e-02, 1.84110373e-01, 2.03298450e-01, 1.06248017e-02,
        6.26387596e-02, 2.69584521e-03, 1.66508090e-02, 1.43672690e-01,
        5.47097996e-02, 2.55312398e-02, 1.64605141e-01, 5.62955923e-02],
       [1.37434555e-02, 1.96335069e-03, 4.00523573e-01, 2.29057600e-03,
        2.46727750e-01, 5.85732982e-02, 8.47513080e-02, 2.02879589e-02,
        1.01112567e-01, 9.16230399e-03, 4.28664908e-02, 1.79973822e-02],
       [2.18669772e-01, 3.15830410e-02, 1.68728128e-01, 5.36756124e-03,
        1.10073902e-01, 2.27926876e-02, 6.52664304e-02, 9.20264497e-02,
        1.33877873e-01, 8.05912092e-02, 3.46168801e-02, 3.64060663e-02],
       [8.40336177e-03, 5.13538765e-03, 1.59197018e-01, 4.66853409e-04,
        3.42670411e-01, 4.29505147e-02, 1.19514473e-01, 5.18207289e-02,
        1.20915033e-01, 5.60224093e-02, 3.50140072e-02, 5.78898229e-02],
       [2.92066745e-02, 4.37004864e-02, 1.46544486e-01, 4.20941189e-02,
        2.65342623e-01, 9.30962712e-03, 1.22668026e-02, 1.76

In [25]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [26]:
tags_df

,X,PRT,VERB,CONJ,NOUN,NUM,ADJ,ADP,DET,ADV,.,PRON
X,0.075167,0.184110,0.203298,0.010625,0.062639,0.002696,0.016651,0.143673,0.054710,0.025531,0.164605,0.056296
PRT,0.013743,0.001963,0.400524,0.002291,0.246728,0.058573,0.084751,0.020288,0.101113,0.009162,0.042866,0.017997
VERB,0.218670,0.031583,0.168728,0.005368,0.110074,0.022793,0.065266,0.092026,0.133878,0.080591,0.034617,0.036406
CONJ,0.008403,0.005135,0.159197,0.000467,0.342670,0.042951,0.119514,0.051821,0.120915,0.056022,0.035014,0.057890
NOUN,0.029207,0.043700,0.146544,0.042094,0.265343,0.009310,0.012267,0.176664,0.013070,0.017232,0.239823,0.004746
NUM,0.211301,0.027663,0.016480,0.013832,0.353738,0.184815,0.033843,0.035315,0.003531,0.002943,0.115362,0.001177
ADJ,0.021664,0.010915,0.011907,0.016868,0.699520,0.020837,0.064660,0.079378,0.004796,0.004465,0.064329,0.000661
ADP,0.034295,0.001496,0.008440,0.000855,0.321688,0.064316,0.105342,0.017094,0.323291,0.014103,0.041132,0.067949
DET,0.046042,0.000242,0.039396,0.000483,0.638550,0.022356,0.203505,0.009305,0.005559,0.012689,0.018127,0.003746
ADV,0.023341,0.013338,0.346782,0.007002,0.032344,0.030343,0.129043,0.118706,0.069356,0.079360,0.135712,0.014672


In [27]:
tags_df.loc['.', :]

X       0.026947
PRT     0.002335
VERB    0.088835
CONJ    0.058385
NOUN    0.220156
NUM     0.081739
ADJ     0.044462
ADP     0.090362
DET     0.173897
ADV     0.052008
.       0.094224
PRON    0.066559
Name: ., dtype: float32

## 3. Viterbi Algorithm



In [29]:
# Viterbi Vannila Algorithm
def Viterbi(words, train_bag = train_tagged_words,tags=T):
    state = []
    T = list(tags)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            word_prob= word_given_tag(words[key], tag)
            emission_p = word_prob[0]/word_prob[1]
            
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        
        
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        
    return list(zip(words, state))



In [30]:
# Viterbi Algorithm with laplacian smoothing
def Laplacian_Viterbi(words, train_bag = train_tagged_words,tags=T):
    state = []
    T = list(tags)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            word_prob= word_given_tag(words[key], tag)
            emission_p = word_prob[0]/word_prob[1]
            if not emission_p:
                emission_p=0.0000000000001
            
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        
        
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
        
    return list(zip(words, state))



In [32]:
#Patterns for regexp Tagger
patterns = [(r'^-?[0-9]+(.[0-9]+)?$', 'NUM'),   # cardinal numbers
      (r'(The|the|A|a|An|an)$', 'DET'),   # articles
      (r'.*able$', 'ADJ'),                # adjectives
      (r'.*ness$', 'NOUN'),               # nouns formed from adjectives
      (r'.*ly$', 'ADV'),                  # adverbs
      (r'.*s$', 'NOUN'),                  # plural nouns
      (r'.*ing$', 'VERB'),                # gerunds
      (r'.*ed$', 'VERB'),                 # past tense verbs
      (r'.*', 'NOUN'),                     # nouns (default)
      (r'.*\*.*\*','X')      
]

In [33]:
#Initialize Regexp Tagger
rex=nltk.RegexpTagger(patterns)

In [34]:
rex.evaluate(test_set)

0.42462262301509507

In [35]:
#Initialize unigram tagger with regexp tagger as backoff
unigram=nltk.UnigramTagger(train_set,backoff=rex)

In [37]:
unigram.evaluate(test_set)

0.9554989217800431

In [53]:
# Viterbi alogrithm with rule based pos-tagging
def Rule_Viterbi(words, train_bag = train_tagged_words,tags=T):
    state = []
    T = list(tags)
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            word_prob= word_given_tag(words[key], tag)
            emission_p = word_prob[0]/word_prob[1]
            
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        if not pmax:
                
                state.append(unigram.tag([words[key]])[0][1])
        else:
            # getting state for which probability is maximum
            state_max = T[p.index(pmax)] 
            state.append(state_max)
        
    return list(zip(words, state))



## 4. Evaluating on Test Set

In [54]:
# list of sents
test_run = test_set

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
#test_run

### Vanilla Viterbi

In [42]:
# tagging the test sentences
start = time.time()
tagged_seq = Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [43]:
print("Time taken in seconds: ", difference)
#print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  29.68366241455078


In [44]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [45]:
accuracy_vannila = len(check)/len(tagged_seq)

In [46]:
accuracy_vannila

0.9119780435208783

### Laplacian Viterbi

In [47]:
# tagging the test sentences
start = time.time()
tagged_seq = Laplacian_Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [48]:
print("Time taken in seconds: ", difference)
#print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  31.630450963974


In [49]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [50]:
accuracy_laplacian = len(check)/len(tagged_seq)

In [51]:
accuracy_laplacian

0.9445206822191727

### Rule-Based Viterbi

In [55]:
# tagging the test sentences
start = time.time()
tagged_seq = Rule_Viterbi(test_tagged_words)
end = time.time()
difference = end-start

In [56]:
print("Time taken in seconds: ", difference)
#print(tagged_seq)
#print(test_run_base)

Time taken in seconds:  31.4149968624115


In [57]:
# accuracy
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 

In [58]:
accuracy_rule = len(check)/len(tagged_seq)

In [59]:
accuracy_rule

0.9580474416781023

### Running on sample sentences

In [60]:
fopen=open('Test_sentences.txt','r')#Open the test sentence file

In [61]:
test_sentence=fopen.read() #Storing all sentences in a variable

In [62]:
test_sentence

"Android is a mobile operating system developed by Google.\nAndroid has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\nGoogle and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\nTwitter is an online news and social networking service on which users post and interact with messages known as tweets.\nBefore entering politics, Donald Trump was a domineering businessman and a television personality.\nThe 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\nThis is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\nShow me the cheapest round trips from Dallas to Atlanta\nI would like to see flights from Denver to Philadelphia.\nShow me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\nNASA invited social media users to experience the launch of ICESAT-2 Satell

In [63]:
words=word_tokenize(test_sentence)#Tokenize the given sentences
words[:20]

['Android',
 'is',
 'a',
 'mobile',
 'operating',
 'system',
 'developed',
 'by',
 'Google',
 '.',
 'Android',
 'has',
 'been',
 'the',
 'best-selling',
 'OS',
 'worldwide',
 'on',
 'smartphones',
 'since']

In [64]:
fopen.close()#Closing the File

### Vanilla Viterbi

In [65]:
start = time.time()
tagged_seq_vannila = Viterbi(words)
end = time.time()
difference = end-start

In [66]:
print(tagged_seq_vannila)
print(difference)

[('Android', 'X'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'X'), ('.', '.'), ('Android', 'X'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'X'), ('worldwide', 'X'), ('on', 'ADP'), ('smartphones', 'X'), ('since', 'ADP'), ('2011', 'X'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'X'), ('.', '.'), ('Google', 'X'), ('and', 'CONJ'), ('Twitter', 'X'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'X'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'X'), ("'s", 'PRT'), ('firehose', 'X'), ('.', '.'), ('Twitter', 'X'), ('is', 'VERB'), ('an', 'DET'), ('online', 'X'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('users', 'NOUN'), ('post', 'NOUN'), ('and', '

## Rule Based Viterbi

In [67]:
start = time.time()
tagged_seq_rule = Rule_Viterbi(words)
end = time.time()
difference = end-start

In [68]:
print(tagged_seq_rule)
print(difference)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'NOUN'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'NOUN'), ('since', 'ADP'), ('2011', 'NUM'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'NUM'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'NUM'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'NOUN'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'NOUN'), ("'s", 'PRT'), ('firehose', 'NOUN'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), (

## Laplacian Viterbi

In [72]:
start = time.time()
tagged_seq_Laplacian = Laplacian_Viterbi(words)
end = time.time()
difference = end-start

In [73]:
print(tagged_seq_Laplacian)
print(difference)

[('Android', 'NOUN'), ('is', 'VERB'), ('a', 'DET'), ('mobile', 'ADJ'), ('operating', 'NOUN'), ('system', 'NOUN'), ('developed', 'VERB'), ('by', 'ADP'), ('Google', 'DET'), ('.', '.'), ('Android', 'NOUN'), ('has', 'VERB'), ('been', 'VERB'), ('the', 'DET'), ('best-selling', 'ADJ'), ('OS', 'NOUN'), ('worldwide', 'NOUN'), ('on', 'ADP'), ('smartphones', 'DET'), ('since', 'ADP'), ('2011', 'DET'), ('and', 'CONJ'), ('on', 'ADP'), ('tablets', 'NOUN'), ('since', 'ADP'), ('2013', 'DET'), ('.', '.'), ('Google', 'NOUN'), ('and', 'CONJ'), ('Twitter', 'NOUN'), ('made', 'VERB'), ('a', 'DET'), ('deal', 'NOUN'), ('in', 'ADP'), ('2015', 'DET'), ('that', 'ADP'), ('gave', 'VERB'), ('Google', 'X'), ('access', 'NOUN'), ('to', 'PRT'), ('Twitter', 'VERB'), ("'s", 'PRT'), ('firehose', 'VERB'), ('.', '.'), ('Twitter', 'NOUN'), ('is', 'VERB'), ('an', 'DET'), ('online', 'NOUN'), ('news', 'NOUN'), ('and', 'CONJ'), ('social', 'ADJ'), ('networking', 'NOUN'), ('service', 'NOUN'), ('on', 'ADP'), ('which', 'DET'), ('user

### Vannila vs Laplacian
Sample Cases where laplacian has tagged correctly over vannila

VANNILA - (ANDROID,X),(NASA,X),(TWITTER,X)


LAPLACIAN -(ANDROID,NOUN),(NASA,NOUN),(TWITTER,NOUN)

### Rule Based Vs Vannila
Sample Cases where Rule-based has tagged correctly over vannila

VANNILA - (domineering, X),(invited,X),(FIFA,X)

Rule-Based - (domineering, VERB),(invited,VERB),(FIFA,NOUN)